In [1]:
import torch
from ultralytics import YOLO

In [2]:
# Check which GPU (if any) accelerator you have access too
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple GPU)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA (not available on Mac)")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using MPS (Apple GPU)


In [3]:
# Load YOLOv8 model
model = YOLO("yolo11s.pt")

100%|██████████| 18.4M/18.4M [00:00<00:00, 45.4MB/s]


In [4]:
# Train the model
results = model.train(data = '../data/data/data.yaml', epochs = 25, batch = 8, device = device)

New https://pypi.org/project/ultralytics/8.3.88 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=../data/data/data.yaml, epochs=25, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

train: Scanning /Users/langtowl/PycharmProjects/gunshot-detection/data/data/train/labels.cache... 720 images, 488 backgrounds, 0 corrupt: 100%|██████████| 720/720 [00:00<?, ?it/s]
val: Scanning /Users/langtowl/PycharmProjects/gunshot-detection/data/data/valid/labels.cache... 180 images, 59 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]

Plotting labels to runs/detect/train/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/25      5.07G      1.217      4.023      1.222         11        640: 100%|██████████| 90/90 [01:24<00:00,  1.06it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:39<00:00,  3.33s/it]

...

                   all        180        174      0.907      0.952      0.952      0.766

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      25/25      8.03G     0.7557      1.097     0.9462          5        640: 100%|██████████| 90/90 [02:32<00:00,  1.70s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:07<00:00,  1.50it/s]


                   all        180        174       0.91      0.937      0.954      0.768

25 epochs completed in 1.299 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 19.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.70 🚀 Python-3.12.4 torch-2.6.0 MPS (Apple M1 Pro)
YOLO11s summary (fused): 238 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:19<00:00,  1.59s/it]


                   all        180        174       0.91      0.937      0.954      0.768
Speed: 3.3ms preprocess, 55.5ms inference, 0.0ms loss, 12.3ms postprocess per image
Results saved to runs/detect/train
